<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2dcc4fe67a8064543dd7e41910b05a76b756ec3e1f1a35aa17bb037210d08be8
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-09 09:36:19
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.37 C
-------------------
Today PnL: -1.63 L (-1.18%)
Current PnL: -30.97 L (-20.0%)
CY Booked + Current PnL: -16.68 L (-10.77%)
-------------------
Total profit:  1.05 L
Total loss:  -32.01 L
-------------------
Total Booked + Current PnL: 10.30 L (8.06%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.41%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 95.60 L (69.62%)
Deployed:  1.28 C
Current:  1.37 C
CAGR/XIRR %: 4.01%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.56,-18.11,22.22,0.09,18338.0,-18247.0,82530.0,91.46,31.0,M-SC,2.19,253.0,-1.00,0.62,8.43,OX40N,NTT,DURABLES
43,ITC,452.00,-0.42,-2.12,12.83,10.44,25133.0,-4245.0,195893.0,-41.56,42.0,X-LC,1.67,5.0,-0.17,1.48,3.32,X40,NTT,FMCG
39,INDIGOPNTS,1408.00,-1.27,-15.57,18.47,0.02,27220.0,-27185.0,147374.0,124.89,44.0,M-SC,16.68,234.0,-1.00,1.11,28.19,OX40N,NTT,PAINTS
50,MASFIN,398.61,-1.14,-5.85,29.63,22.05,27334.0,-5730.0,92250.0,-18.70,47.0,H-SC,6.87,164.0,-0.21,0.70,34.09,XR,ATH,FINANCE
35,ICICIGI,2252.93,-1.24,3.33,16.32,20.19,29396.0,5802.0,180122.0,-19.08,39.0,X-MC,5.94,68.0,0.20,1.36,18.95,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,ICICIPRULI,790.00,0.53,2.70,27.52,30.97,49953.0,4776.0,181514.0,-21.49,53.0,X-MC,2.02,75.0,0.10,1.37,15.60,X40,ATH,INSURANCE
33,HINDUNILVR,2922.00,0.32,-9.70,25.88,13.67,60075.0,-24933.0,232130.0,-16.44,31.0,X-LC,2.99,14.0,-0.42,1.75,9.59,XY25,NTT,FMCG
27,GLAXO,3466.20,-0.05,2.14,36.23,39.15,73744.0,4260.0,203544.0,-18.01,49.0,X-MC,11.66,60.0,0.06,1.54,31.76,X40,ATH,PHARMA
19,COLPAL,3726.84,-0.13,-18.60,73.85,41.51,158312.0,-48995.0,214370.0,-3.00,43.0,X-MC,7.30,61.0,-0.31,1.62,1.99,XY25,ATH,FMCG
34,HONAUT,58357.33,-0.19,-16.04,70.71,43.32,96689.0,-26132.0,136740.0,-27.81,37.0,X-SC,3.76,90.0,-0.27,1.03,4.99,X40N,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-4.76,-48.61,163.24,35.27,95249.0,-55200.0,58349.0,-59.76,18.0,L-SC,12.34,270.0,-0.58,0.44,42.81,XR,NTT,HOTELS
28,GREENPANEL,537.00,-3.85,-40.37,140.05,43.14,152887.0,-73912.0,109166.0,144.83,21.0,M-SC,8.70,240.0,-0.48,0.82,2.69,XY24,NTT,MISC
65,SIEMENS,4671.50,-3.66,-13.91,45.79,25.51,73360.0,-25885.0,160210.0,-1.60,48.0,H-LC,3.70,49.0,-0.35,1.21,18.06,AR,ATH,ELECTRICAL
5,ASIANPAINT,3460.25,-3.54,-1.27,22.50,20.95,55929.0,-3194.0,248574.0,-2.32,52.0,X-LC,19.07,31.0,-0.06,1.88,33.92,X40,ATH,PAINTS
0,5PAISA,593.00,-3.13,-44.23,102.46,12.90,124843.0,-96650.0,121846.0,79.09,36.0,H-SC,11.50,161.0,-0.77,0.92,1.31,OX40N,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.73,-9.82,130.55,107.91,173819.0,-14500.0,133144.0,-26.63,23.0,M-SC,11.95,216.0,-0.08,1.01,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.27,-15.57,18.47,0.02,27220.0,-27185.0,147374.0,124.89,44.0,M-SC,16.68,234.0,-1.00,1.11,28.19,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.56,-18.11,22.22,0.09,18338.0,-18247.0,82530.0,91.46,31.0,M-SC,2.19,253.0,-1.00,0.62,8.43,OX40N,NTT,DURABLES
66,SIS,528.00,-0.86,-25.82,63.47,21.26,52277.0,-28667.0,82365.0,1940.93,42.0,H-SC,4.66,166.0,-0.55,0.62,11.28,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.64,-26.44,54.25,13.47,107620.0,-71289.0,198378.0,-69.67,24.0,H-SC,3.25,158.0,-0.66,1.50,0.87,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-1.1,6.03,62.48,72.27,112250.0,10210.0,179658.0,-8.8,67.0,M-LC,2.23,99.0,0.09,1.36,12.57,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.10,6.03,62.48,72.27,112250.0,10210.0,179658.0,-8.80,67.0,M-LC,2.23,99.0,0.09,1.36,12.57,XR,NTT,IT
38,INDIAMART,4810.62,-0.64,-2.93,112.97,106.72,135243.0,-3620.0,119716.0,-51.96,33.0,H-SC,1.19,139.0,-0.03,0.90,19.76,AR,ATH,MISC
37,IEX,219.00,-0.70,-4.37,55.42,48.63,105971.0,-8739.0,191215.0,-36.48,47.0,H-SC,14.09,136.0,-0.08,1.44,6.68,XR,NTT,MISC
25,FINCABLES,1641.55,-0.73,-9.82,130.55,107.91,173819.0,-14500.0,133144.0,-26.63,23.0,M-SC,11.95,216.0,-0.08,1.01,0.00,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-1.00,-3.47,40.51,35.63,81948.0,-7282.0,202292.0,-17.18,33.0,H-MC,3.18,119.0,-0.09,1.53,12.62,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-1.29,-24.39,96.34,48.45,146053.0,-48904.0,151602.0,-29.75,9.0,X-MC,11.30,64.0,-0.33,1.15,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.42,-39.07,121.90,35.20,95567.0,-50272.0,78398.0,2.82,22.0,X-SC,15.33,92.0,-0.53,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.70,-14.37,41.99,21.58,50387.0,-20142.0,119997.0,-56.81,22.0,X-MC,5.51,56.0,-0.40,0.91,6.24,X40N,ATH,IT
46,JSWINFRA,342.00,-1.54,-12.77,32.10,15.23,56097.0,-25590.0,174757.0,-25.37,23.0,X-MC,7.98,66.0,-0.46,1.32,15.36,X40N,NTT,REALTY
75,TMPV,600.00,-1.28,-18.52,74.49,42.18,167768.0,-51188.0,225222.0,-26.74,25.0,X-LC,5.11,3.0,-0.31,1.70,0.00,XY24,NTT,AUTO


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-1.76,-22.08,96.99,53.50,228044.0,-66630.0,235121.0,-62.23,31.0,X-MC,1.24,58.0,-0.29,1.78,4.43,XY24,NTT,FMCG
43,ITC,452.0,-0.42,-2.12,12.83,10.44,25133.0,-4245.0,195893.0,-41.56,42.0,X-LC,1.67,5.0,-0.17,1.48,3.32,X40,NTT,FMCG
15,CAMS,4762.0,-1.39,-80.53,541.00,24.79,265258.0,-202825.0,49031.0,-81.54,38.0,X-SC,1.80,86.0,-0.76,0.37,19.86,X40N,NTT,MISC
3,ACC,3906.0,-1.19,-25.42,120.09,64.15,213124.0,-60481.0,177470.0,-56.31,34.0,X-SC,1.82,82.0,-0.28,1.34,0.00,XY24,BTT,CEMENT
36,ICICIPRULI,790.0,0.53,2.70,27.52,30.97,49953.0,4776.0,181514.0,-21.49,53.0,X-MC,2.02,75.0,0.10,1.37,15.60,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-1.19,-25.42,120.09,64.15,213124.0,-60481.0,177470.0,-56.31,34.0,X-SC,1.82,82.0,-0.28,1.34,0.0,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-1.29,-24.39,96.34,48.45,146053.0,-48904.0,151602.0,-29.75,9.0,X-MC,11.30,64.0,-0.33,1.15,0.0,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.42,-39.07,121.90,35.20,95567.0,-50272.0,78398.0,2.82,22.0,X-SC,15.33,92.0,-0.53,0.59,0.0,X40,NTT,FOOTWEAR
31,HAVELLS,2069.16,-0.78,-10.60,48.19,32.48,141304.0,-34768.0,293223.0,-15.19,34.0,X-MC,5.19,67.0,-0.25,2.22,0.0,X40,ATH,ELECTRICAL
54,PGHH,17907.65,-1.23,-6.12,42.47,33.76,80071.0,-12285.0,188535.0,-33.72,33.0,X-MC,5.11,57.0,-0.15,1.42,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-2.15,-32.52,111.68,42.84,48990.0,-21140.0,43866.0,-55.92,34.0,X-SC,37.13,83.0,-0.43,0.33,0.00,XY24,NTT,MISC
15,CAMS,4762.00,-1.39,-80.53,541.00,24.79,265258.0,-202825.0,49031.0,-81.54,38.0,X-SC,1.80,86.0,-0.76,0.37,19.86,X40N,NTT,MISC
59,RELAXO,1176.00,-0.63,-48.80,197.46,52.31,146764.0,-70834.0,74326.0,-45.50,29.0,X-SC,6.19,91.0,-0.48,0.56,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.42,-39.07,121.90,35.20,95567.0,-50272.0,78398.0,2.82,22.0,X-SC,15.33,92.0,-0.53,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.70,-14.37,41.99,21.58,50387.0,-20142.0,119997.0,-56.81,22.0,X-MC,5.51,56.0,-0.40,0.91,6.24,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-1.25,-13.16,37.35,19.28,112215.0,-45513.0,300443.0,-24.27,61.0,X-LC,5.77,1.0,-0.41,2.27,11.07,X40,ATH,IT
41,INFY,2275.00,-1.14,8.40,42.87,54.87,145407.0,26293.0,339181.0,-15.65,62.0,X-LC,3.37,2.0,0.18,2.56,17.38,X40,BTT,IT
75,TMPV,600.00,-1.28,-18.52,74.49,42.18,167768.0,-51188.0,225222.0,-26.74,25.0,X-LC,5.11,3.0,-0.31,1.70,0.00,XY24,NTT,AUTO
81,VBL,671.64,-1.86,-6.87,45.68,35.67,134368.0,-21692.0,294150.0,-17.96,46.0,X-LC,4.86,4.0,-0.16,2.22,6.03,X40N,ATH,FMCG
43,ITC,452.00,-0.42,-2.12,12.83,10.44,25133.0,-4245.0,195893.0,-41.56,42.0,X-LC,1.67,5.0,-0.17,1.48,3.32,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,-1.14,-5.85,29.63,22.05,27334.0,-5730.0,92250.0,-18.70,47.0,H-SC,6.87,164.0,-0.21,0.70,34.09,XR,ATH,FINANCE
13,BSOFT,831.70,-0.81,-23.00,95.83,50.79,103782.0,-32355.0,108298.0,-1.04,67.0,H-SC,5.80,151.0,-0.31,0.82,26.09,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.27,-15.57,18.47,0.02,27220.0,-27185.0,147374.0,124.89,44.0,M-SC,16.68,234.0,-1.00,1.11,28.19,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-3.54,-1.27,22.50,20.95,55929.0,-3194.0,248574.0,-2.32,52.0,X-LC,19.07,31.0,-0.06,1.88,33.92,X40,ATH,PAINTS
68,STARHEALTH,761.00,-0.73,-11.30,66.03,47.26,164641.0,-31781.0,249342.0,20.96,33.0,H-SC,13.14,171.0,-0.19,1.88,33.59,XY24,NTT,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.81,-23.00,95.83,50.79,103782.0,-32355.0,108298.0,-1.04,67.0,H-SC,5.80,151.0,-0.31,0.82,26.09,XR,ATH,IT
32,HCLTECH,1922.01,-1.00,8.49,14.97,24.73,39290.0,20533.0,262457.0,13.18,66.0,X-LC,5.79,7.0,0.52,1.98,25.02,X40,ATH,IT
84,WIPRO,420.00,-1.10,6.03,62.48,72.27,112250.0,10210.0,179658.0,-8.80,67.0,M-LC,2.23,99.0,0.09,1.36,12.57,XR,NTT,IT
5,ASIANPAINT,3460.25,-3.54,-1.27,22.50,20.95,55929.0,-3194.0,248574.0,-2.32,52.0,X-LC,19.07,31.0,-0.06,1.88,33.92,X40,ATH,PAINTS
73,TCS,4389.97,-1.25,-13.16,37.35,19.28,112215.0,-45513.0,300443.0,-24.27,61.0,X-LC,5.77,1.0,-0.41,2.27,11.07,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.88
1,25,45.94
2,50,76.87


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.64
MC    30.40
LC    24.95
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.21
X40      22.69
X40N     10.86
XR        9.98
XY25      9.45
AR        9.35
OX40N     7.69
SR        1.02
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.32
X-MC    23.67
X-LC    21.33
M-SC    11.98
X-SC     6.98
H-MC     4.97
M-MC     1.40
L-SC     1.36
M-LC     1.36
H-LC     1.21
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.54,-8.40,43.45
IT,12.93,-17.75,81.41
FINANCE,9.46,-23.84,76.16
ELECTRICAL,6.27,-13.10,54.52
PAINTS,6.13,-13.81,30.72
MISC,6.10,-60.34,126.02
INSURANCE,4.61,-3.47,39.93
PHARMA,4.14,-2.11,34.78
AUTO,3.41,-27.16,81.67


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3301943.0,21
XR,1342121.0,13
AR,1333675.0,10
X40,1034729.0,14
X40N,1011612.0,9
OX40N,773195.0,10
XY25,390170.0,6
SR,286378.0,2
MH,86071.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3760728.0,25
M-SC,1470693.0,15
X-MC,1351647.0,16
X-SC,994357.0,8
X-LC,938227.0,11
H-MC,421663.0,3
L-SC,281683.0,3
M-LC,112250.0,1
H-LC,73360.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1322590.0      6
           AR         907792.0      5
M-SC       XY24       832639.0      6
H-SC       XR         804083.0      7
X-SC       X40N       508711.0      3
X-MC       X40        468452.0      7
           XY24       400783.0      3
X-LC       X40        377974.0      5
H-SC       OX40N      353814.0      4
M-SC       OX40N      329757.0      5
X-SC       XY24       297343.0      3
X-MC       X40N       291665.0      4
H-SC       SR         286378.0      2
X-LC       XY24       249991.0      2
H-MC       AR         223066.0      2
X-LC       X40N       211236.0      2
H-MC       XY24       198597.0      1
L-SC       XR         192059.0      2
X-MC       XY25       190747.0      2
X-SC       X40        188303.0      2
M-SC       XR         178840.0      2
           AR         129457.0      2
M-LC       XR         112250.0      1
X-LC       XY25        99026.0      2
L-SC       OX40N       89624.0      1
H-SC       MH          86071.0      1
H-LC       AR          73360.0      1
M-MC       XY25        59395.0      1
L-MC       XR          54889.0      1
L-LC       XY25        41002.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
